In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

print(sys.path)

import numpy as np
import pandas as pd
import cvxpy as cp

from src.setup_problem import Setup
from src.format_winston_data import WinstonDataFormatter_v2
from src.format_winston_data import generate_and_save_unsupervised_data
from src.misc import *

['/home/onoue/ws/lukasiewicz_1/notebooks/winston_2', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.11/site-packages', '/home/onoue/ws/lukasiewicz_1']


# 0. データの準備

In [3]:
file_name_list = os.listdir("./winston_pca_2/")
file_path_list = [os.path.join("./winston_pca_2", file_name) for file_name in file_name_list]

dfs = [pd.read_csv(file, index_col=0) for file in file_path_list]
df_data = pd.concat(dfs, ignore_index=True)

df_data

,animal,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,cheetah,-20.060238,-11.480357,15.623350,-6.305073,-6.696852,4.830261,-5.379857,2.432294,9.030500,...,0.125542,-0.080214,0.926857,-0.314778,-0.374476,-0.507503,0.318324,-0.128268,-0.045080,-0.206593
1,cheetah,-48.204088,18.591528,4.103486,2.440145,-10.566893,-14.992575,-2.351199,9.304754,-7.887125,...,-1.832162,-0.673048,1.983817,2.164188,0.928383,1.989706,0.480698,-1.720412,0.499422,-4.106438
2,cheetah,-13.188533,-23.096405,2.572728,-3.706356,2.869635,-3.104971,3.141484,1.542608,5.318094,...,-0.348495,-0.930289,0.061565,0.067395,-0.050295,-0.080876,0.121771,-0.493991,-0.457423,-0.195045
3,cheetah,-20.250505,-9.332849,-3.731214,-2.956475,-4.907211,-2.487805,-10.028531,1.898730,1.671651,...,-0.171502,-0.323352,0.199332,0.127779,-0.849878,-0.229659,-0.313232,0.213185,-1.336118,-0.310712
4,cheetah,-20.059971,-1.582472,-7.138032,-11.185771,18.417116,-1.680940,-6.145799,-0.107130,3.890427,...,1.681730,0.600582,2.152980,-2.025689,-0.199217,-0.165662,-0.718164,2.490273,0.523311,1.913032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,ostrich,31.291656,6.482612,-11.086591,-10.524747,4.290951,6.700974,21.181147,-8.942196,5.719353,...,-0.332219,-0.871387,-0.577859,-0.421761,0.225885,-0.944522,0.130902,-0.473095,0.188853,-0.326705
6496,ostrich,-6.391303,-17.068252,-1.496960,-14.264796,3.159807,-2.084721,8.642888,-2.086789,-8.758540,...,-2.444848,-1.743830,-1.234302,-1.398103,-3.151357,0.797208,-0.660319,0.189967,0.322385,-0.457151
6497,ostrich,-33.814540,10.456383,15.762074,6.935928,-1.560411,-3.322494,-3.718956,10.833294,6.694091,...,0.151620,0.986486,-1.042699,0.025971,-0.971935,-0.679953,0.292943,0.210233,0.598297,-0.516183
6498,ostrich,34.769526,21.752479,-3.132908,5.004080,3.544195,14.021153,-4.892887,1.884487,3.299574,...,-0.115667,-0.087014,0.304643,0.442750,0.269512,-0.045713,0.555787,-0.068632,0.019545,-0.239788


In [4]:
df_labels = pd.read_csv('./labels.csv', index_col=0).fillna(-1)
display(df_labels.head())
print(df_labels.shape)

,albatross,cheetah,ostrich,penguin,zebra
hair(x),-1.0,1.0,-1.0,-1.0,1.0
mammal(x),-1.0,1.0,-1.0,-1.0,1.0
milk(x),-1.0,1.0,-1.0,-1.0,-1.0
feather(x),1.0,-1.0,1.0,1.0,-1.0
bird(x),1.0,-1.0,1.0,1.0,-1.0


(31, 5)


In [6]:
def random_split_by_label(df, label_column='animal', train_size=0.8, sample_size=None, selected_labels=None):
    # 選択されたラベルのみを対象にする
    if selected_labels is not None:
        df = df[df[label_column].isin(selected_labels)]

    # サンプルサイズがNoneの場合、元のデータをそのまま返す
    if sample_size is None:
        return df.reset_index(drop=True), pd.DataFrame()  # テストデータは空のDataFrameを返す

    # 各ラベル毎に指定の個数をランダムに抜き出す
    sampled_dataframes_train = []
    sampled_dataframes_test = []
    unique_labels = df[label_column].unique()

    for label in unique_labels:
        label_data = df[df[label_column] == label]
        # サンプリング
        sample_data = label_data.sample(n=sample_size, random_state=42)
        # インデックスをリセット
        sample_data = sample_data.reset_index(drop=True)
        # 訓練データとテストデータに分割
        split_idx = int(train_size * len(sample_data))
        train_data = sample_data.loc[:split_idx - 1, :]
        test_data = sample_data.loc[split_idx:, :]
        # リストに追加
        sampled_dataframes_train.append(train_data)
        sampled_dataframes_test.append(test_data)

    # 訓練データを結合
    result_df_train = pd.concat(sampled_dataframes_train, ignore_index=True)
    # テストデータを結合
    result_df_test = pd.concat(sampled_dataframes_test, ignore_index=True)

    return result_df_train, result_df_test

# 使用例
original_df = df_data.copy()
selected_labels = ['cheetah', 'albatross']
train_data, test_data = random_split_by_label(original_df, train_size=0.8, sample_size=125, selected_labels=selected_labels)

print("Train Data:")
print(train_data)

print("\nTest Data:")
print(test_data)


Train Data:
        animal          0          1          2          3          4  \
0      cheetah  15.402662  -3.807138   3.632292  32.764571  -5.664949   
1      cheetah -50.593476  -7.922209 -11.033482   6.115892   2.230875   
2      cheetah -10.398780  49.083416  12.765650   3.406599   2.639825   
3      cheetah  -0.934629  -2.057008  -0.887415  -7.865467  -7.703866   
4      cheetah  10.387551   2.727248   0.374913   3.122029  -1.818549   
..         ...        ...        ...        ...        ...        ...   
195  albatross -10.554250 -10.180245  15.334719   1.620547  -4.822957   
196  albatross   2.097137  31.900102  -2.181612   9.192555   0.074148   
197  albatross -34.750480   5.682939  -2.522935   1.560119   4.492572   
198  albatross -51.186763  -6.981031  -5.796971 -21.319474   7.340640   
199  albatross   6.656991 -26.175263 -13.083918   0.192902 -10.015450   

             5          6          7         8  ...       190       191  \
0     2.876672   3.040756  -8.643766

In [9]:
display(train_data.head())
print(train_data.shape)

,animal,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,cheetah,15.402662,-3.807138,3.632292,32.764571,-5.664949,2.876672,3.040756,-8.643766,7.808112,...,-4.031143,0.502019,0.630617,0.088934,-0.005006,2.169094,-1.383973,-0.150240,-2.225031,-0.375203
1,cheetah,-50.593476,-7.922209,-11.033482,6.115892,2.230875,-2.827979,-8.589891,3.639380,-0.459745,...,0.152746,0.734783,0.300104,-0.220096,-0.999787,-0.102783,-0.280356,-0.086377,-0.098555,-0.002831
2,cheetah,-10.398780,49.083416,12.765650,3.406599,2.639825,-1.695449,-15.687951,-9.690763,-6.096559,...,-0.634467,-0.049114,-0.563111,0.451740,0.678812,0.386950,-0.803237,-0.082105,-0.050870,-0.782950
3,cheetah,-0.934629,-2.057008,-0.887415,-7.865467,-7.703866,-2.916751,1.805643,-4.821153,0.240899,...,-0.383386,0.378471,1.924444,1.428540,0.363502,0.191559,-1.472842,-2.003535,0.022197,-0.438671
4,cheetah,10.387551,2.727248,0.374913,3.122029,-1.818549,-7.941755,-2.613647,-8.021543,-0.653920,...,-1.009989,-1.500397,-0.375630,-1.125995,-0.887603,1.666718,-0.122017,-0.161629,0.449841,0.364122


(200, 201)


In [10]:
display(test_data.head())
print(test_data.shape)

,animal,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,cheetah,8.292308,30.280626,-2.508785,-12.189169,1.316509,1.452295,0.662887,-2.593478,7.865164,...,0.127151,0.234225,0.026973,0.601735,0.706454,-0.376523,0.212934,0.234671,-0.050421,0.712837
1,cheetah,1.065279,-3.032873,1.046417,1.808066,-13.745040,-10.891609,5.774404,8.382609,1.120438,...,-0.974262,-1.424370,-0.630855,1.112381,-1.126797,0.259563,1.669981,0.930642,0.928045,0.305732
2,cheetah,28.422621,8.662941,-2.154647,-5.213169,6.219972,-2.105719,1.345773,-0.472902,2.814426,...,-0.915350,0.553261,-0.368945,-0.154758,0.038356,0.347901,-0.105149,-0.141596,0.331212,-0.150608
3,cheetah,21.324583,-25.946052,-4.888448,2.588104,2.207624,5.901980,6.962974,3.425877,2.180212,...,0.737898,-0.838535,0.200311,-0.297229,0.007516,-0.514221,-0.165555,0.113626,-0.361711,0.476461
4,cheetah,-6.533477,-7.372995,-12.838421,5.062599,-2.775070,6.158497,-0.320145,-2.181440,-8.993230,...,-0.369579,-1.692979,-0.007752,0.756700,-2.113139,0.483583,-0.284576,-1.362919,0.625620,1.438815


(50, 201)


In [15]:
data_formatter_1 = WinstonDataFormatter_v2(train_data, df_labels)
data_formatter_1.format_data()
save_dir_path = os.path.join(project_dir_path, "inputs/winston_pca_100_two_animals_v2")

path_tmp = os.path.join(save_dir_path, "train")

if not os.path.exists(path_tmp):
    os.makedirs(path_tmp)

data_formatter_1.save_as_csv(path_tmp)
generate_and_save_unsupervised_data(path_tmp, data_num=30, data_dim=200)

path_tmp = os.path.join(save_dir_path, "test")

if not os.path.exists(path_tmp):
    os.makedirs(path_tmp)

data_formatter_2 = WinstonDataFormatter_v2(test_data, df_labels)
data_formatter_2.format_data()
data_formatter_2.save_as_csv(path_tmp)

Done!


In [16]:
train_data.to_csv(os.path.join(save_dir_path, "train_base.csv"))
test_data.to_csv(os.path.join(save_dir_path, "test_base.csv"))

# 1. 

---

- 動物が cheetah と albatross のみ
- 各 predicate に対してデータ数は 100 個

In [27]:
data_dir_path = os.path.join(project_dir_path, 'inputs/winston_pca_100_two_animals_v2')

train_data_dir_path = os.path.join(data_dir_path, "train")

file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules_2']
}

problem_instance = Setup(train_data_dir_path, file_names_dict)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')

Loading data ...
Done in 0.603381872177124 seconds! 

Loading rules ...
Done in 0.0004291534423828125 seconds! 

Identifying predicates ...
Done in 0.0013113021850585938 seconds! 

Constructing objective function ...
Done in 1.2085447311401367 seconds! 

Constructing constraints ...
Done in 7.379160404205322 seconds! 

All done
                                     CVXPY                                     
                                     v1.3.2                                    


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.11/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


(CVXPY) Nov 12 03:35:15 PM: Your problem has 12463 variables, 22380 constraints, and 0 parameters.
(CVXPY) Nov 12 03:35:17 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 12 03:35:17 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 12 03:35:17 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 12 03:35:20 PM: Compiling problem (target solver=ECOS).
(CVXPY) Nov 12 03:35:20 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> ECOS
(CVXPY) Nov 12 03:35:20 PM: Applying reduction Dcp2Cone
(CVXPY) Nov 12 03:35:28 PM: Applying reduction CvxAttr2Constr
(CVXPY) Nov 12 03:35:32 PM: A

In [28]:
def test_trained_predicate(predicates_dict, test_data_dict):
    result_dict = {}
    p_names = predicates_dict.keys()

    for p_name in p_names:
        pred_vals = []
        preds = []

        p = predicates_dict[p_name]
        test_data = test_data_dict[p_name]

        if isinstance(test_data, pd.DataFrame):
            test_data = np.array(test_data)

        cnt = 0

        for data in test_data:

            x, ans = data[:-1], data[-1]
            pred_val = p(x).value
            pred_vals.append(pred_val)

            if (pred_val >= 0.5 and ans == 1) or (pred_val < 0.5 and ans == -1):
                cnt += 1

            pred = (pred_val >= 0.5 and ans == 1) or (pred_val < 0.5 and ans == -1)
            preds.append(pred)

        p_arr = np.hstack([test_data, 
                           np.array(pred_vals).reshape(-1,1), 
                           np.array(preds).reshape(-1, 1)])
        
        # p_df = pd.DataFrame(p_arr, columns=['r', 'g', 'b', 'Ans', 'pred_val', 'pred'])

        result_dict[p_name] = p_arr

        print(cnt)
        print(f'Accuracy of {p_name}: {cnt / len(test_data)}')

    return result_dict

In [31]:
test_data_dir_path = os.path.join(data_dir_path, "test")
file_path_1 = os.path.join(test_data_dir_path, "L_albatross(x).csv")
file_path_2 = os.path.join(test_data_dir_path, "L_cheetah(x).csv")

In [33]:
test_data = {
    'albatross(x)': np.array(pd.read_csv(file_path_1, index_col=0)),
    'cheetah(x)': np.array(pd.read_csv(file_path_2, index_col=0))
}

In [35]:
p_dict = problem_instance.predicates_dict
selected_predicates = ['cheetah(x)', 'albatross(x)']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}

In [36]:
res_dict = test_trained_predicate(selected_p_dict, test_data)

25
Accuracy of cheetah(x): 0.5
25
Accuracy of albatross(x): 0.5


In [37]:
res_dict

{'cheetah(x)': array([[ 8.29230805e+00,  3.02806264e+01, -2.50878517e+00, ...,
          1.00000000e+00,  6.81793554e-01,  1.00000000e+00],
        [ 1.06527921e+00, -3.03287332e+00,  1.04641680e+00, ...,
          1.00000000e+00, -1.01313778e+00,  0.00000000e+00],
        [ 2.84226211e+01,  8.66294058e+00, -2.15464666e+00, ...,
          1.00000000e+00, -1.67796147e-02,  0.00000000e+00],
        ...,
        [-6.46389085e+01, -6.14163022e+00,  2.75040551e+00, ...,
         -1.00000000e+00,  5.47786290e+00,  0.00000000e+00],
        [ 4.40781719e+01, -1.14120938e+01,  1.40741224e+01, ...,
         -1.00000000e+00, -2.79395426e-01,  1.00000000e+00],
        [-2.95621189e+01,  5.87864528e+00, -9.76806457e+00, ...,
         -1.00000000e+00, -2.13455081e+00,  1.00000000e+00]]),
 'albatross(x)': array([[  8.29230805,  30.28062643,  -2.50878517, ...,  -1.        ,
           0.31820645,   1.        ],
        [  1.06527921,  -3.03287332,   1.0464168 , ...,  -1.        ,
           2.01313778

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

data_dir_path = os.path.join(project_dir_path, 'inputs/winston_pca_100_two_animals_v2')

train_data = pd.read_csv(os.path.join(data_dir_path, "train_base.csv"), index_col=0)
test_data = pd.read_csv(os.path.join(data_dir_path, "test_base.csv"), index_col=0)

# 特徴量とラベルを分ける
X_train = train_data.drop('animal', axis=1)
X_test = test_data.drop('animal', axis=1)
y_train = train_data['animal']
y_test = test_data['animal']

# Random Forestモデルの作成と学習
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimatorsは適宜変更可能
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [21]:
print(accuracy)
print()
print(classification_rep)

0.54

              precision    recall  f1-score   support

   albatross       0.54      0.52      0.53        25
     cheetah       0.54      0.56      0.55        25

    accuracy                           0.54        50
   macro avg       0.54      0.54      0.54        50
weighted avg       0.54      0.54      0.54        50



In [22]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

data_dir_path = os.path.join(project_dir_path, 'inputs/winston_pca_100_two_animals_v2')

train_data = pd.read_csv(os.path.join(data_dir_path, "train_base.csv"), index_col=0)
test_data = pd.read_csv(os.path.join(data_dir_path, "test_base.csv"), index_col=0)

# 特徴量とラベルを分ける
X_train = train_data.drop('animal', axis=1)
X_test = test_data.drop('animal', axis=1)
y_train = train_data['animal']
y_test = test_data['animal']

# SVMモデルの作成と学習
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [23]:
print(accuracy)
print()
print(classification_rep)

0.4

              precision    recall  f1-score   support

   albatross       0.40      0.40      0.40        25
     cheetah       0.40      0.40      0.40        25

    accuracy                           0.40        50
   macro avg       0.40      0.40      0.40        50
weighted avg       0.40      0.40      0.40        50



In [24]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

data_dir_path = os.path.join(project_dir_path, 'inputs/winston_pca_100_two_animals_v2')

train_data = pd.read_csv(os.path.join(data_dir_path, "train_base.csv"), index_col=0)
test_data = pd.read_csv(os.path.join(data_dir_path, "test_base.csv"), index_col=0)

# 特徴量とラベルを分ける
X_train = train_data.drop('animal', axis=1)
X_test = test_data.drop('animal', axis=1)
y_train = train_data['animal']
y_test = test_data['animal']

# SVMモデルの作成と学習
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [25]:
print(accuracy)
print()
print(classification_rep)

0.66

              precision    recall  f1-score   support

   albatross       0.67      0.64      0.65        25
     cheetah       0.65      0.68      0.67        25

    accuracy                           0.66        50
   macro avg       0.66      0.66      0.66        50
weighted avg       0.66      0.66      0.66        50

